Import stuff

In [66]:
import os
import re
import string

import requests
from bs4 import BeautifulSoup

import nltk
nltk.download('punkt')
nltk.download('stopwords')

import pymorphy2

from collections import defaultdict as dd
from operator import itemgetter

import math

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/Alexey.Zhebel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Alexey.Zhebel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Get the first page

In [2]:
URL = "https://www.jetbrains.com/help/idea/"
page = ""

In [3]:
source = requests.get(URL + page)
print(source.text[:500])

<!DOCTYPE html SYSTEM "about:legacy-compat"><html lang="en-US" data-colors-preset="contrast" data-primary-color="#307FFF"><head><link data-react-helmet="true" rel="canonical" href="https://www.jetbrains.com/help/idea/discover-intellij-idea.html"><meta charset="UTF-8">  <meta name="built-on" content="2021-12-21T11:59:50.784961"><meta name="build-number" content="8116">       <script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start': new Date().getTime(),event:'gtm.js'});var f=d.getElement


Get soup from source

In [4]:
soup = BeautifulSoup(source.text, "html.parser")

print(soup.prettify()[:500])

# with open("page_src.html", "w") as file:
#     file.write(soup.prettify())

<!DOCTYPE html SYSTEM "about:legacy-compat">
<html data-colors-preset="contrast" data-primary-color="#307FFF" lang="en-US">
 <head>
  <link data-react-helmet="true" href="https://www.jetbrains.com/help/idea/discover-intellij-idea.html" rel="canonical"/>
  <meta charset="utf-8"/>
  <meta content="2021-12-21T11:59:50.784961" name="built-on"/>
  <meta content="8116" name="build-number"/>
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start': new Date().getTime(),event:'gtm.js'});v


Get text from soup

Only the topic title, chapter and procedure headings, and paragraphs

In [5]:
article = soup.find("article")

text = ""

for tag in article.find_all(re.compile("^h[1-6]$|^p$")):
    text += tag.get_text() + "\n"

print(repr(text[:500]))

'IntelliJ\xa0IDEA overview\nIntelliJ\xa0IDEA is an Integrated Development Environment (IDE) for JVM languages designed to maximize developer productivity. It does the routine and repetitive tasks for you by providing clever code completion, static code analysis, and refactorings, and lets you focus on the bright side of software development, making it not only productive but also an enjoyable experience.\nMulti-platform\nIntelliJ\xa0IDEA is a cross-platform IDE that provides consistent experience on Windows,'


Tokenize text

In [11]:
tokens = nltk.tokenize.word_tokenize(text.lower().translate(str.maketrans('', '', string.punctuation)))

print(tokens[:100])

['intellij', 'idea', 'overview', 'intellij', 'idea', 'is', 'an', 'integrated', 'development', 'environment', 'ide', 'for', 'jvm', 'languages', 'designed', 'to', 'maximize', 'developer', 'productivity', 'it', 'does', 'the', 'routine', 'and', 'repetitive', 'tasks', 'for', 'you', 'by', 'providing', 'clever', 'code', 'completion', 'static', 'code', 'analysis', 'and', 'refactorings', 'and', 'lets', 'you', 'focus', 'on', 'the', 'bright', 'side', 'of', 'software', 'development', 'making', 'it', 'not', 'only', 'productive', 'but', 'also', 'an', 'enjoyable', 'experience', 'multiplatform', 'intellij', 'idea', 'is', 'a', 'crossplatform', 'ide', 'that', 'provides', 'consistent', 'experience', 'on', 'windows', 'macos', 'and', 'linux', 'see', 'install', 'intellij', 'idea', 'for', 'osspecific', 'instructions', 'see', 'intellij', 'idea', 'keyboard', 'shortcuts', 'for', 'instructions', 'on', 'how', 'to', 'choose', 'the', 'right', 'keymap', 'for', 'your', 'operating', 'system']


Lemmatize?

In [7]:
word = pymorphy2.MorphAnalyzer().parse("teachers")

print(word)

[Parse(word='teachers', tag=OpencorporaTag('LATN'), normal_form='teachers', score=1.0, methods_stack=((LatinAnalyzer(score=0.9), 'teachers'),))]


Frequency

In [12]:
word_count = dd(int)
stop_words = nltk.corpus.stopwords.words("english")

for word in tokens:
    if word not in stop_words:
        word_count[word] += 1

sorted_word_count = sorted(word_count.items(), key=itemgetter(1), reverse=True)

for word, freq in sorted_word_count[:10]:
    print('\t'.join((word, str(freq))))

intellij	145
idea	142
ide	47
jetbrains	41
install	36
code	34
version	34
account	34
run	32
click	30


Get the next page

In [9]:
next_page = soup.find("a", {"class":"navigation-links__next"}).get("href")

print(next_page)

product-educational-tools.html


Let's try to scrape 5 first topics and count words

In [10]:
page = next_page

for i in range(5):
    source = requests.get(URL + page)
    soup = BeautifulSoup(source.text, "html.parser")
    article = soup.find("article")

    for tag in article.find_all(re.compile("^h[1-6]$|^p$")):
        text += tag.get_text() + "\n"

    page = soup.find("a", {"class":"navigation-links__next"}).get("href")

print(text)

IntelliJ IDEA overview
IntelliJ IDEA is an Integrated Development Environment (IDE) for JVM languages designed to maximize developer productivity. It does the routine and repetitive tasks for you by providing clever code completion, static code analysis, and refactorings, and lets you focus on the bright side of software development, making it not only productive but also an enjoyable experience.
Multi-platform
IntelliJ IDEA is a cross-platform IDE that provides consistent experience on Windows, macOS, and Linux.
See Install IntelliJ IDEA for OS-specific instructions.
See IntelliJ IDEA keyboard shortcuts for instructions on how to choose the right keymap for your operating system, and learn the most useful shortcuts.
Supported languages
Development of modern applications involves using multiple languages, tools, frameworks, and technologies. IntelliJ IDEA is designed as an IDE for JVM languages but numerous plugins can extend it to provide a polyglot experience.
JVM languages
Use Intel

Let's make a corpus: a list of documents (each document is text extracted from a topic)

In [14]:
corpus = []

page = ""

for i in range(5):
    text = ""
    source = requests.get(URL + page)
    soup = BeautifulSoup(source.text, "html.parser")
    article = soup.find("article")

    for tag in article.find_all(re.compile("^h[1-6]$|^p$")):
        text += tag.get_text() + "\n"

    corpus.append(text)

    page = soup.find("a", {"class":"navigation-links__next"}).get("href")

print(len(corpus))

IntelliJ IDEA overview
IntelliJ IDEA is an Integrated Development Environment (IDE) for JVM language


In [42]:
token_corpus = []

for document in corpus:
    tokens = nltk.tokenize.word_tokenize(document.lower().translate(str.maketrans('', '', string.punctuation)))
    token_corpus.append(tokens)

print(len(token_corpus[0]))

2207


In [43]:
print(token_corpus[0][:20])

['intellij', 'idea', 'overview', 'intellij', 'idea', 'is', 'an', 'integrated', 'development', 'environment', 'ide', 'for', 'jvm', 'languages', 'designed', 'to', 'maximize', 'developer', 'productivity', 'it']


In [44]:
clean_token_corpus = []
stop_words = nltk.corpus.stopwords.words("english")\

for document in token_corpus:
    for token in document:
        if token in stop_words:
            document.remove(token)
    clean_token_corpus.append(document)

print(len(clean_token_corpus[0]))


1583


In [45]:
print(clean_token_corpus[0][:20])

['intellij', 'idea', 'overview', 'intellij', 'idea', 'integrated', 'development', 'environment', 'ide', 'jvm', 'languages', 'designed', 'maximize', 'developer', 'productivity', 'does', 'routine', 'repetitive', 'tasks', 'providing']


In [75]:
term_frequencies = []

for document in clean_token_corpus:
    tf = dd(int)
    for word in document:
        tf[word] += 1 / len(document)

    term_frequencies.append(tf)

# sorted_word_count = sorted(word_count.items(), key=itemgetter(1), reverse=True)
#
# for word, freq in sorted_word_count[:10]:
#     print('\t'.join((word, str(freq))))

print(len(term_frequencies))

5


In [76]:
# print(term_frequencies[0])

sorted_frequency_table = sorted(term_frequencies[0].items(),
                                key=itemgetter(1), reverse=True)

for word, freq in sorted_frequency_table[:10]:
    print('\t'.join((word, str(freq))))

intellij	0.030953885028427004
idea	0.0296904611497157
the	0.028427037271004395
code	0.01895135818066961
a	0.01137081490840177
your	0.010739102969046116
can	0.010107391029690462
ide	0.008843967150979154
see	0.008843967150979154
also	0.007580543272267847


In [81]:
for document in term_frequencies:
    for term in document:
        docs_with_term = 0
        for doc in term_frequencies:
            if term in doc:
                docs_with_term += 1
        document[term] = document[term] * math.log(len(term_frequencies) / docs_with_term)
        # print(term, document[term])

# print(term_frequencies[0])

sorted_frequency_table = sorted(term_frequencies[4].items(),
                                key=itemgetter(1), reverse=True)

for word, freq in sorted_frequency_table[:10]:
    print('\t'.join((word, str(freq))))

website	0.19689058978708146
token	0.15145429983621653
authorization	0.13630886985259485
register	0.12116343986897324
eaprelated	0.09087257990172992
logged	0.07572714991810826
url	0.07572714991810826
redirected	0.06058171993448662
floating	0.06058171993448662
proxy	0.06058171993448662


In [94]:
source = requests.get("https://www.jetbrains.com/help/idea/getting-started.html")

print(source.text[:100])

soup = BeautifulSoup(source.text, "html.parser")

with open("page_src.html", "w") as file:
    file.write(soup.prettify())

print(soup.find("article"))

<!DOCTYPE html SYSTEM "about:legacy-compat"><html lang="en-US" data-colors-preset="contrast" data-pr
None
